In [7]:
using JSON

# Specify the file name and location
filename = "tiny.json"

# Read the JSON file
data = JSON.parsefile(filename)

# Extract the details from the JSON
# For example, extract all shops
linear_model = true  # Set to false if you don't want a linear model
shops = data["shops"]
println("Shops:")
println(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]

for vehicle in vehicles
    if linear_model
        # Set all vehicles to regular if linear_model is true
        println("Vehicle Type: regular")
    else
        # Otherwise, print the actual vehicle type
        println("Vehicle ID: ", vehicle["id"], ", Type: ", vehicle["type"])
    end
end
for constraint in constraints
    println("Constraint: ", constraint)
end
for parameter in parameters
    println("Parameter: ", parameter)
end
for shop in shops
    println("Shop: ", shop)
end



Shops:
Any[Dict{String, Any}("name" => "body", "resequencing_lag" => 1), Dict{String, Any}("name" => "paint", "resequencing_lag" => 2), Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)]
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Constraint: Dict{String, Any}("max_vehicles" => 4, "cost" => 2, "min_vehicles" => 2, "id" => 1, "vehicles" => Any[1, 2, 4], "type" => "batch_size", "shop" => "body")
Constraint: Dict{String, Any}("cost" => 3, "partition" => Any[Any[1, 2], Any[3, 4, 5]], "id" => 2, "type" => "lot_change", "shop" => "paint")
Constraint: Dict{String, Any}("max_vehicles" => 2, "cost" => 2, "id" => 3, "window_size" => 3, "vehicles" => Any[1, 2, 5], "type" => "rolling_window", "shop" => "assembly")
Constraint: Dict{String, Any}("max_vehicles" => 3, "cost" => 3, "min_vehicles" => 2, "id" => 4, "vehicles" => Any[1, 2, 3], "type" => "batch_size", "shop" => "assembly")
Constraint: Dict{String, Any}("max_ve

In [8]:
resequencing_cost = parameters["resequencing_cost"]
number_of_shops = length(shops)    
lags = 
for shop in shops 
    println("Shop: ", shop)
end
println("Number of shops: ", number_of_shops)
number_of_vehicles = length(vehicles)
println("number_of_vehicles, ",number_of_vehicles)

Shop: Dict{String, Any}("name" => "body", "resequencing_lag" => 1)
Shop: Dict{String, Any}("name" => "paint", "resequencing_lag" => 2)
Shop: Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)
Number of shops: 3
number_of_vehicles, 5


In [9]:
using JuMP
using Gurobi

# Initialize an empty dictionary to store the mapping
shop_map = Dict{String, Vector{Any}}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
lag = Dict{String, Integer}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
# Populate the shop_map with shop names as keys and empty arrays as values
for shop in shops
    shop_map[shop["name"]] = []
    lag[shop["name"]] = shop["resequencing_lag"]
end
n = number_of_vehicles
println("n: ", n)
# Initialize counters or variables if needed

# Assume `model` is defined somewhere before this loop (like in JuMP)
# Example: model = Model(Ipopt.Optimizer)

# Iterate through constraints and associate them with the corresponding shop in shop_map

for constraint in constraints
    name = constraint["shop"]
    if haskey(shop_map, name)
        push!(shop_map[name], constraint)
    else
        println("Warning: Shop name $name in constraints does not exist in shops.")
    end
end

┌ Warning: Module JLLWrappers with build ID fafbfcfd-3e00-7921-0000-8c72a4ea3f6c is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2541


n: 5

┌ Warning: Module JLLWrappers with build ID fafbfcfd-3e00-7921-0000-8c72a4ea3f6c is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2541


In [10]:
function permutation(x,n)
    permutation = [0 for i in 1:n]  # Initialize the permutation array
    
    for i in 1:n
        for j in 1:n
            if x[i, j] == 1
                permutation[i] = j
            end
        end
    end
    return permutation
end


permutation (generic function with 1 method)

In [11]:
for (shop, constraints) in shop_map
    model = Model(GLPK.Optimizer)  # Create the model with a solver
    total_cost = 0
    println("Shop: $shop")
    x = @variable(model, [1:n, 1:n], Bin)
    for v in 1:n
        @constraint(model, sum(x[v, t] for t in 1:n) == 1)
        @constraint(model, sum( x[t, v] for t in 1:n) == 1)
    end
    for constraint in constraints
        println("    Shop: ", constraint["shop"])
        if constraint["type"] == "lot_change"
            println("Type: lot_change")
            cl = constraint["cost"]
            k = length(constraint["partition"])
            Ul = constraint["partition"]
            println("UL is ", Ul)
            u = @variable(model, [1:n, 1:k], Bin)
            f = @variable(model, [1:n], Bin)
            
            for t in 1:n
                for o in 1:k
                    sum = 0
                    list = Ul[o]
                    for v in list
                        sum += x[v, t]
                    end
                    @constraint(model, u[t, o] == sum)
                end
            end
            
            for t in 1:n-1
                for o in 1:k
                    @constraint(model, f[t] >= u[t, o] - u[t+1, o])
                end
            end
            total_cost += cl * sum(f)
        end
        if constraint["type"] == "rolling_window"
            println("Type: rolling_window")
            q = @variable(model, [1:n], Bin)
            z = @variable(model, [1:n], Int)
            @constraint(model, z .>= 0)
            Vr = constraint["vehicles"]
            Mr = constraint["max_vehicles"]
            wr = constraint["window_size"]
            cr = constraint["cost"]

            for t in 1:n
                sum = 0
                for v in Vr
                    @constraint(model, q[t] >= x[v, t])
                    sum += x[v, t]
                end
                @constraint(model, q[t] <= sum)
            end

            for t in 1:n
                sum = 0
                for t_prime in t:min(t+wr-1, n)
                    sum += q[t_prime]
                end
                @constraint(model, z[t] >= sum - Mr)
            end
            for t in 1:max(n-wr+1,1) 
                total_cost += cr * z[t]
            end
        end
        if constraint["type"] == "batch_size"
            println("    Type: batch_size")
            mb = constraint["min_vehicles"]
            Mb = constraint["max_vehicles"]
            Vb = constraint["vehicles"]
            cb = constraint["cost"]

            function gamma(t)
                return cb * max(0, max(mb-t, t-Mb))^2
            end
            p = @variable(model, [1:n], Bin)
            m = @variable(model, [1:n, 1:n], Bin)
            l = @variable(model, [1:n, 1:n], Bin)
            j = @variable(model, [1:n, 1:n], Bin)
            g = @variable(model, [1:n, 1:n], Bin)
            for t in 1:n
                sum = 0
                for v in Vb
                    @constraint(model, p[t] >= x[v, t])
                    sum += x[v, t]
                end
                @constraint(model, p[t] <= sum)
            end

            for t in 1:n
                for t_prime in t:n
                    if t >= 2
                        @constraint(model, j[t, t_prime] >= 1 - p[t])
                    end
                    for k in t_prime:t
                        @constraint(model, m[t, t_prime] >= p[t])
                    end
                    if t_prime <= n-1
                        @constraint(model, l[t, t_prime] >= 1 - p[t_prime])
                    end
                end
            end
            for t in 1:n
                for t_prime in t:n
                    @constraint(model, g[t, t_prime] >= m[t, t_prime] + l[t, t_prime] + j[t, t_prime] - 2)
                end
            end
            for t in 1:n-1 
                for t_prime in t:n
                    total_cost += gamma(t_prime-t+1) * g[t, t_prime]
                end
            end
        end 
    end
    @objective(model, Min, total_cost)
    # Optimize model
    optimize!(model)
    # Output results
    if termination_status(model) == MOI.OPTIMAL
        println(termination_status(model))
        x_values = value.(x)
        println("Total cost for shop $shop: $(objective_value(model))")
        println("Optimized values of x:")
        println(x_values)
        println("Permutation: ", permutation(x_values, n))
    else
        println("Solver termination status: ", termination_status(model))
    end
    println("next shop")
end


Shop: body
    Shop: body
    Type: batch_size
OPTIMAL
Total cost for shop body: 0.0
Optimized values of x:
[0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0]
Permutation: [2, 3, 1, 4, 5]
next shop
Shop: assembly
    Shop: assembly
Type: rolling_window
    Shop: assembly
    Type: batch_size
    Shop: assembly
    Type: batch_size
OPTIMAL
Total cost for shop assembly: 0.0
Optimized values of x:
[0.0 0.0 0.0 1.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0]
Permutation: [4, 1, 5, 2, 3]
next shop
Shop: paint
    Shop: paint
Type: lot_change
UL is Any[Any[1, 2], Any[3, 4, 5]]
OPTIMAL
Total cost for shop paint: 3.0
Optimized values of x:
[0.0 1.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0]
Permutation: [2, 1, 5, 3, 4]
next shop


$
\text{The algorithm converges for the small instances but still do not know if it will for the big instances}. \newline
 \text {The solution also given by this approach is far from being a good solution, we need to discuss what to do after this to solve it.}
$

In [13]:
for (shop, constraints) in shop_map
    model = Model(GLPK.Optimizer)  # Create the model with a solver
    total_cost = 0
    
    println("Shop: $shop")
    println("Lag: ", lag[shop])
    x = @variable(model, [1:n, 1:n], Bin)
    for v in 1:n
        @constraint(model, sum(x[v, t] for t in 1:n) == 1)
        @constraint(model, sum( x[t, v] for t in 1:n) == 1)
    end

    for constraint in constraints
        println("    Shop: ", constraint["shop"])
        
        if constraint["type"] == "lot_change"
            println("Type: lot_change")
            cl = constraint["cost"]
            k = length(constraint["partition"])
            println("k is ", k)
            Ul = constraint["partition"]
            println("UL is ", Ul)
            u = @variable(model, [1:n, 1:k], Bin)
            f = @variable(model, [1:n], Bin)
            
            for t in 1:n
                for o in 1:k
                    sum = 0
                    list = Ul[o]
                    for v in list
                        sum += x[v, t]
                    end
                    @constraint(model, u[t, o] == sum)
                end
            end
            
            for t in 1:n-1
                s = 0
                for o in 1:k
                    @constraint(model, f[t] >= u[t, o] - u[t+1, o])
                    s += u[t, o] - u[t+1, o]
                end
            end
            total_cost += cl * sum(f)
        end
    end
    @objective(model, Min, total_cost)
    # Optimize model
    optimize!(model)

    # Output results
    if termination_status(model) == MOI.OPTIMAL
        println(termination_status(model))
        x_values = value.(x)
        println("Total cost for shop $shop: $(objective_value(model))")
        println("Optimized values of x:")
        println(x_values)
        println("Permutation: ", permutation(x_values, n))
    else
        println("Solver termination status: ", termination_status(model))
    end
    println("next shop")
end


Shop: body
Lag: 1
    Shop: body
OPTIMAL
Total cost for shop body: 0.0
Optimized values of x:
[1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0]
Permutation: [1, 2, 3, 4, 5]
next shop
Shop: assembly
Lag: 0
    Shop: assembly
    Shop: assembly
    Shop: assembly
OPTIMAL
Total cost for shop assembly: 0.0
Optimized values of x:
[1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0]
Permutation: [1, 2, 3, 4, 5]
next shop
Shop: paint
Lag: 2
    Shop: paint
Type: lot_change
k is 2
UL is Any[Any[1, 2], Any[3, 4, 5]]
OPTIMAL
Total cost for shop paint: 3.0
Optimized values of x:
[0.0 1.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0]
Permutation: [2, 1, 5, 3, 4]
next shop
